## Portfolio Optimization

Portfolio optimization models look for the optimal way to make investments. Usually investors expect 
either a maximum return for a given level of risk or a given return for a minimum risk so these models
are typically based on two criteria: maximization of the expected return and/or minimization of the risk.

There is a variety of measures of risk and the most popular is the variance in return. 

### Some Notation

* expected return: 
    $$\mathbb{E}(R_{p}) = \sum _{i}w_{i} \mathbb{E}(R_{i})$$
  where $R_{p}$ is the return on the portfolio, $R_{i}$ is the return on
  asset $i$ and $w_{i}$ is the weighting of component asset $i$
  (that is, the proportion of asset $i$ in the portfolio) and
  $\sum_{i}w_i = 1$ and $0 \le w_i \le 1$;
* portfolio return variance:
  $$ \sigma _{p}^{2} = \sum _{i}\sum _{j}w_{i}w_{j}\sigma _{ij}$$ 
  where
  $\Sigma = \sigma _{ij}=\sigma _{i}\sigma _{j}\rho _{ij}$ is the (sample)
  covariance of the periodic returns on the two assets, $\rho_{ij}$ is the correlation coefficient. In matrix notation:
  $$\sigma_p^2 = \mathbf{w^T}\Sigma\mathbf{w}$$
  where $\mathbf{w} = (w_1, w_2,\ldots,w_N)$ is the vector of weights.
* portfolio return volatility (standard deviation):
  $$ \sigma _{p}= \sqrt{\sigma _{p}^{2}}$$

### The Markowitz Mean/Variance Portfolio Model
The portfolio model, introduced by Markowitz, assumes an investor has two considerations when constructing an investment portfolio: expected return and variance in return (i.e., risk, since it measures the variability in realized return around the expected return). 

The Markowitz model requires two major kinds of information: 

* the estimated expected return for each candidate investment;
* the covariance matrix of returns. 

The latter characterizes not only the individual variability of the return on each investment, but also how each investment’s return tends to move with other investments. 

Throughout this lesson we will use real market data stored in [portfolio_data.csv](https://drive.google.com/file/d/1srCzNlKVY_LHRpkaKoUynnmI0KImfT6Y/view?usp=sharing).
The sample includes, for each entry, a date and the corresponding closing price of five company stocks:

In [208]:
import pandas as pd

df = pd.read_csv("portfolio_data.csv", index_col="date")
print (df.head())

                 AAPL     AMZN     FB    GOOG       NFLX
date                                                    
2014-03-27  71.865678  338.470  60.97  558.46  52.025714
2014-03-28  71.785450  338.290  60.01  559.99  51.267143
2014-03-31  71.769404  336.365  60.24  556.97  50.290000
2014-04-01  72.425937  342.990  62.62  567.16  52.098571
2014-04-02  72.546280  341.960  62.72  567.00  51.840000


<img src="portfolio_sample.png">

With $\tt{pandas}$ the main characteristics of these time series can be easily computed (e.g. daily returns, covariance matrix):

In [220]:
# returns daily and annualized
daily_returns = df.pct_change()
returns = daily_returns.mean()*252
print (returns)

AAPL    0.239188
AMZN    0.415127
FB      0.263797
GOOG    0.172818
NFLX    0.528046
dtype: float64


In [210]:
# covariance
covariance = daily_returns.cov()*252
print (covariance)

          AAPL      AMZN        FB      GOOG      NFLX
AAPL  0.051902  0.025037  0.025737  0.022454  0.027760
AMZN  0.025037  0.085839  0.041025  0.039501  0.048412
FB    0.025737  0.041025  0.069550  0.036127  0.044528
GOOG  0.022454  0.039501  0.036127  0.051797  0.040390
NFLX  0.027760  0.048412  0.044528  0.040390  0.178298


In our sample correlations are rather small and all the stocks are positively correlated.

Simulating a large number of set of weights to construct portfolios with the five stocks shown before, we can see which is the distribution of these portfolios in terms of return and volatility. In this case no attempt of any optimization whatsoever has been made.

In principle investors may use short sales in their portfolios (a portfolio is short in those stocks with negative weights). Although short selling extends the set of possible portfolios we are not going to consider them here.

<img src="return_variance.png" width=500>

### Optimization

Markowitz model states that the weights $w_i$ should be chosen such that the portfolio has the minimum volatility (variance). So the application of Markovitz model reduces to an optimization problem

$$\underset{\mathbf{w}}{\min}\{\sigma_P^2\}= \underset{\mathbf{w}}{\min}\{\mathbf{w^T}\Sigma\mathbf{w}\}$$

with the constraint $\sum_{i}w_i = 1$ and $0 \le w_i \le 1$.

With have already seen how to solve minimization problems in $\tt{python}$ so we just need to repeat the usual steps seen before.

In [222]:
# markowitz
import numpy as np
from scipy.optimize import minimize

def sum_weights(w): 
    return np.sum(w) - 1

def markowitz(w, cov):
    return w.T.dot(cov.dot(w))

num_assets = 5
constraints = ({'type': 'eq', 'fun': sum_weights},) 
bounds = tuple((0, 1) for asset in range(num_assets))
weights = [1./num_assets for _ in range(num_assets)]

opts = minimize(markowitz, weights, args=(covariance,),
                bounds=bounds, constraints=constraints)
print (opts)

     fun: 0.03607029963784209
     jac: array([0.0723022 , 0.07233166, 0.0718961 , 0.07199311, 0.07223676])
 message: 'Optimization terminated successfully.'
    nfev: 63
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([0.44544146, 0.06252825, 0.12333117, 0.3662157 , 0.00248342])


In [223]:
print ("Expected portfolio return: {:.3f}".format(np.dot(opts.x, returns)))

Expected portfolio return: 0.230


The solution recommends about 44% of the portfolio be invested in AAPL, about 7% in AMZN, 12% in FB and so on...
The expected return is about 23%, with a variance of about 0.036 or,
equivalently, a standard deviation of 0.19.

In this example we based the model simply on statistical data derived from daily returns. However it could be possible, rather than use historical series for estimating the
expected return of an asset, to base this estimate on information about its expected future performance.

### Efficient Frontier and Parametric Analysis
There is no precise way for an investor to determine the “correct” trade off between risk and return. If an investor wants a higher expected return, she generally has to “pay for it” with higher risk. 
Thus, one is frequently interested in looking at the relative distribution of the two. 

In finance terminology, we
would like to trace out the **efficient frontier** of return and risk. 
To determine it we need to solve for the minimum variance
portfolio over a range of values for the expected return.

In [224]:
def efficient_frontier(w, asset_returns, target_return): 
    portfolio_return = asset_returns.dot(w) 
    return (portfolio_return - target_return)

results = []
bounds = tuple((0, 1) for asset in range(num_assets))
    
for eff in np.arange(0.20, 0.45, 0.005):
    constraints = ({'type': 'eq', 'fun': efficient_frontier, 
                    'args':(returns, eff,)},
                   {'type': 'eq', 'fun': sum_weights})
    weights = [1./num_assets for _ in range(num_assets)]
    opts = minimize(markowitz, weights, args=(covariance,),
                    bounds=bounds, constraints=constraints) 
    
    results.append((np.sqrt(opts.x.T.dot(covariance.dot(opts.x))),
                    np.sum(returns*opts.x))) 

<img src="efficient_frontier.png">

### Criticisms to Markowitz Model

Despite the significant utility of the Markowitz theory, there are some major limitations in this model:

* it is difficult to forecast asset returns with accuracy using historical data. As return estimations have a much larger impact on the asset allocations, small changes in return assumptions can lead to inefficient portfolios. Therefore, the model tends to lead to highly concentrated portfolios (out-of sample weights) that do not offer as much diversification benefits;
* the model assumes that asset correlations are linear. In reality, asset correlations move dynamically, changing with the market cycles. During the global financial crisis, asset correlations approached almost 1, so if anything, diversification seemed to have insignificant impacts on the portfolios;
* last but not the least, the model assumes normality in return distributions. Therefore, it does not factor in extreme market moves which tend to make returns distributions either skewed, fat tailed or both.

### Portfolios with a Risk-Free Asset
When one of the asset of the portfolio is risk free, then the efficient frontier has a particularly simple form: a line, the *capital allocation line* (CAL). 

The slope of the this line measures the trade off between risk and return: a higher slope means that investors receive a higher expected
return in exchange for taking on more risk. 
The CAL aids investors in choosing how much to invest in a risk-free asset and one or more risky assets.

The simplest example is a portfolio containing two assets: one risk-free (e.g. treasury bill) and one risky (e.g. a stock).

Assume that the expected return of the treasury bill is $\mathbb{E}[R_f] = 3\%$ and its risk is 0%. Further, assume that the expected return of the stock is $\mathbb{E}[R_r] = 10\%$ and its standard deviation is $\sigma_r = 20\%$. The question that needs to be answered for any individual investor is how much to invest in each of these assets.

The expected return ($\mathbb{E}[R_p]$) of this portfolio is calculated as follows:

$$\mathbb{E}[R_p] = \mathbb{E}[R_f]\cdot w_f + \mathbb{E}[R_r] \cdot ( 1 − w_f )$$
where $w_f$ is the relative allocation to the risk-free asset.

The calculation of risk for this portfolio is simple because the standard deviation of the treasury bill is 0%. Thus, risk is calculated as:

$$\sigma_p = ( 1 − w_f ) \cdot \sigma_r$$

In this very simple example, if an investor were to invest 100% into the risk-free asset ($w_f = 1$),
the expected return would be 3% and the risk of the portfolio would be 0%. Likewise, investing
100% into the stock ($w_f = 0$) would give an investor an expected return of 10% and a portfolio risk
of 20%. If the investor allocated 25% to the risk-free asset and 75% to the risky asset, the portfolio
expected return and risk calculations would be:

$$\mathbb{E}[R_p] = ( 3\% \cdot 25\% ) + ( 10\% \cdot 75\% ) = 0.75\% + 7.5\% = 8.25\%$$

$$\sigma_p = 75\%\cdot 20\% = 15\%$$

Applying the same steps to our example we can consider an additional risk-free asset with an expected return of 10% and repeat the minimisation to determine the efficient frontier of
the resulting portfolio. Notice how the objective function is almost the same while the constraint on the target return now includes also the risk-free asset.

In [281]:
num_assets = 6
    
def markowitz_with_rf(w, cov):
    return w[:-1].T.dot(cov.dot(w[:-1]))

def efficient_frontier_with_rf(w, asset_returns, target_return, 
                               risk_free): 
    portfolio_return = np.sum(asset_returns*w[:-1]) + risk_free*w[5] 
    return (portfolio_return - target_return)

rf_asset_return = 0.10
results_rf = []
bounds = tuple((0, 1) for asset in range(num_assets))
for eff in np.arange(0.10, 0.45, 0.01):
    constraints = ({'type': 'eq', 'fun': efficient_frontier_with_rf, 
                    "args":(returns, eff, rf_asset_return)},
                    {'type': 'eq', 'fun': sum_weights})
    weights = [1./num_assets for _ in range(num_assets)]
    opts = minimize(markowitz_with_rf, weights, 
                    args=(covariance),
                    bounds=bounds, constraints=constraints)
    results_rf.append((np.sqrt(opts.x[:-1].T.dot(covariance.dot(opts.x[:-1]))), 
                     np.sum(returns*opts.x[:-1])+opts.x[5]*rf_asset_return))

<img src="cal.png">

The efficient frontier has become a straight line, tangent to the
frontier of the risky assets only. 

When the target is 10% the entire investment is allocated to the
risk-free asset, as the target increases the fraction of the risky assets grows proportionally to the volatility.
It is important to note that in general the relative proportions devoted to risky investments do not change. Only the allocation between the risk-free asset and the risky assets changes.

### The Sharpe Ratio
The goal of an investor who is seeking to earn the highest possible expected return for any 
level of volatility is to find the portfolio that generates the steepest possible line
when combined with the risk-free investment. The slope of this line is called the 
*Sharpe ratio* of the portfolio.

For some portfolio $p$ of risky assets let

* $R_p$ its expected return;
* $\sigma_p$ its standard deviation in return;
* $r_0$ the return of a reference risk-free asset.

A plausible single measure (as opposed to the two measures, risk and return) of attractiveness of a portfolio $p$ is the Sharpe ratio defined as

$$ \cfrac{R_p - r_0}{\sigma_p} $$

In words, it measures how much additional return we achieved for the additional risk we took on, relative to putting all our money in a risk-free asset.
The portfolio that maximizes this ratio has a certain well-defined appeal. Suppose:

* $R_\textrm{target}$ our desired target return;
* $w_p$ the fraction of our wealth we place in portfolio $p$ (the rest placed in the risk-free asset).

To meet our return target, we must have:

$$ ( 1 - w_p ) * r_0 + w_p * R_p = R_\textrm{target} $$

The standard deviation of our total investment is: $w_p\cdot \sigma_p$.
Solving for $w_p$ in the return constraint, we get:

$$ w_p = \cfrac{( R_\textrm{target} – r_0)}{( R_p – r_0)} $$

Thus, the standard deviation of the portfolio is:

$$ w_p\cdot \sigma_p = \Big[\cfrac{( R_\textrm{target} – r_0)}{(R_p – r_0)}\Big]\cdot \sigma_p $$

Minimizing the portfolio standard deviation means:

$$ \min(\Big[\cfrac{( R_\textrm{target} – r_0)}{(R_p – r_0)}\Big]\cdot \sigma_p)\implies\max(\cfrac{R_p – r_0}{\sigma_p}) $$

So, regardless of our risk/return preference, the money we invest in risky assets should be invested in the risky portfolio that maximizes the Sharpe ratio since it is the one that minimize the variance at the same time .

In [234]:
num_assets = 5
rf_asset_return = 0.10

def negativeSharpeRatio(w, asset_returns, rf_asset_return, cov): 
    p_ret = np.sum(asset_returns*w)
    p_var = np.sqrt(w.T.dot(cov.dot(w)))
    ratio = -(p_ret - rf_asset_return) / p_var
    return ratio

constraints = ({'type': 'eq', 'fun': sum_weights})
bounds = tuple((0, 1) for asset in range(num_assets))
weights = [1./num_assets for _ in range(num_assets)]
opts = minimize(negativeSharpeRatio, weights, 
                args=(returns, rf_asset_return, covariance),
                bounds=bounds, constraints=constraints)
print (opts)

     fun: -1.259317843963551
     jac: array([-0.37875988, -0.37936528, -0.26915939,  0.02855793, -0.37932482])
 message: 'Optimization terminated successfully.'
    nfev: 42
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([1.19754268e-01, 5.43974232e-01, 4.33680869e-19, 6.41847686e-17,
       3.36271500e-01])


In [235]:
print ("Sharpe ratio: ", -opts.fun)

Sharpe ratio:  1.259317843963551


<img src="sharpe_ratio.png">

Notice that the relative proportions of the stocks are the same as in the previous case where we explicitly included the risk-free asset (0.12, 0.54, 0., 0., 0.33).

So the optimization using the Sharpe ratio gives us a portfolio that is
on the minimum volatility efficient frontier, and gives the maximum
return relative to putting all our money in the risk-free asset.

Usually, any Sharpe ratio greater than 1.0 is considered acceptable to good by investors. A ratio higher than 2.0 is rated as very good. A ratio of 3.0 or higher is considered excellent. A ratio under 1.0 is sub-optimal.

## Capital Asset Pricing Model

The Capital Asset Pricing Model (CAPM) describes the relationship between expected return of assets and *systematic risk* of the market.

Indeed we can divide a security’s total risk into unsystematic risk, the portion peculiar to the company that can be diversified away, and systematic risk, the nondiversifiable portion that is related to the movement of the stock market and is therefore unavoidable.

The assumption of CAPM is that investors are risk averse and want to maximize return. This
notion implies that investors demand compensation for taking on risk, which in the financial market means that higher-risk securities are priced to yield higher expected returns than lower risk securities.

CAPM states that the expected return of an asset is equal to the risk-free return plus a risk premium. Mathematically, we can summarize CAPM with the following formula

$$r_i = r_f + \beta_i(r_m-r_f)$$
where:
* $r_i$ is the expected return of the $i^{th}$ security;
* $r_f$ is the risk-free rate with zero standard deviation (an example of a risk-free asset includes Treasury Bills as they are backed by the U.S. government);
* $r_m - r_f$ is the risk premium ($r_m$ denotes the market return including all securities in the market, it can be represented with an index like S\&P 500);
* $\beta_i$ is a measure of $i^{th}$ asset volatility in relation to the overall market. 
$\beta$ is used in the CAPM to describe the relationship between systematic risk, or market risk, and its expected return.
	
Therefore the key point in CAPM is the determination of $\beta$ which can be achieved with the measurement of the slope of the *regression line*, of the market vs individual stock return distribution.

Given two sets of measurements $X$ and $y$ the linear regression determines the parameter $\alpha$ and $\beta$ such that

$$y=\beta X + \alpha$$

by minimizing the sum of the squared differences between the predicted and true $y$ values.

<img src="linear_regression.png">

In our case the regressed line estimates the stock returns given the global market returns and in particular 

$$\beta \approx \cfrac{\textrm{cov}(X,y)}{\textrm {var}(X)}$$

so provides insights about how *volatile*, or how risky, a stock is relative to the rest of the market.
In CAPM $\beta$ calculation is used to help investors understand whether a stock moves in the same direction as the rest of the market but for it to provide any useful insight, the market that is used as a benchmark should be related to the stock.

Those who use CAPM pick individual stocks or portfolios, and compare them to different indexes. The point is to find stocks that have high $\beta$, and portfolios that have high $\alpha$. High $\beta$ values mean that the stock fares better than index, so those stocks have a chance at beating the market. $\alpha$ values above zero mean that your portfolio gives positive return no matter what the market does.

This risk vs expected return relationship is called the security market line (SML). 

<img src ="sel.png">

In the freely competitive financial markets described by CAPM, no security can sell for long at prices low enough to yield more than its appropriate return on the SML. The security would then be very attractive compared with other securities of similar risk, and investors would bid its price up until its expected return fell to the appropriate position on the SML. Conversely, investors would sell off any stock selling at a price high enough to put its expected return below its appropriate position. The resulting reduction in price would continue until the stock’s expected return rose to the level justified by its systematic risk.

In order to see an example of CAPM application we use [capm.csv](https://drive.google.com/file/d/1G4U8foyhq9agGPs8cg83-aY4qU8K3jJI/view?usp=sharing) file which contains the historical series of S&P500 and of some stocks.
As usual with $\tt{pandas}$ we can inspect the file and compute some useful quantities like daily and annualized expected returns.

In [260]:
import pandas as pd

capm = pd.read_csv("capm.csv", index_col='date')
print (capm.head())
daily_returns = capm.pct_change()*100

                 AAPL     AMZN          BA    GOOG         IBM        MGM  \
date                                                                        
27/03/2014  71.865678  338.470  111.078480  558.46  167.346339  24.629343   
28/03/2014  71.785450  338.290  112.205402  559.99  167.892905  24.609560   
31/03/2014  71.769404  336.365  113.133986  556.97  169.691285  25.578908   
01/04/2014  72.425937  342.990  115.586169  567.16  171.463219  26.241625   
02/04/2014  72.546280  341.960  115.676323  567.00  170.625738  26.409777   

                    T    TSLA        SP500  
date                                        
27/03/2014  28.801410  207.32  1849.040039  
28/03/2014  28.892032  212.37  1857.619995  
31/03/2014  28.892032  208.45  1872.339966  
01/04/2014  28.908509  216.97  1885.520020  
02/04/2014  29.139184  230.29  1890.900024  


### Calculate $\beta$ and CAPM for each Stock
To recap $\beta$ is the slope of the regression line of the market return vs stock return plot (and $\alpha$ is the intercept of this line with the $y$ axis).

These quantities can be calculated with $\tt{numpy.polyfit}$ passing as inputs the market and the stock return lists. 

<img src="capm_fit.png">

With the daily returns and $\beta$ of each stock, we can calculate the capital asset pricing model. First, we can calculate the average daily return of the market and annualize this return by multiplying it by the number of trading days in a year.
Assuming a risk-free rate of 1%, we can then calculate CAPM using its definition.

In [266]:
rm = np.mean(daily_returns.iloc[1:]['SP500'])*252 
rf = 0.01
betas = {}
alphas = {}
ERs = {}
for i, k in enumerate(capm.columns):
    if k == "SP500": 
        continue
    betas[k], alphas[k] = np.polyfit(daily_returns['SP500'][1:], daily_returns[k][1:], 1)
    ERs[k] = rf + (betas[k] * (rm-rf))
    
for k, v in ERs.items(): 
    print ("{:4}: {:4.1f}%".format(k, v))

AAPL: 10.3%
AMZN: 10.8%
BA  : 10.3%
GOOG: 10.7%
IBM :  8.7%
MGM : 13.8%
T   :  5.9%
TSLA: 11.9%


Now that we have the $\beta$ of each individual stock we can calculate the CAPM for a portfolio made of the same stocks (we assume equal weights for simplicity).
It is enough indeed to perform a weighted sum of of the expected return according to the model of each stock.

In [268]:
 print ("{:.1f}%".format(sum(ERs.values())*1/len(ERs)))

10.3%


The expected return of the portfolio is roughly 10\% and this is what an investor should expect according to CAPM.

### Criticism to CAPM 
As we have seen the whole model is about plotting a line in a scatter plot. It’s not a very complex model. Assumptions under the model are even more simplistic. For example:

* expect that all investors are rational and they avoid risk;
* everyone have full information about the market;
* everyone have similar investment horizons and expectations about future movements;
* stocks are all correctly priced.

Moreover, this is a model from the 1960s. Market dynamics were different back then. And of course, this is a retrospective model. We cannot know how future stock prices move and how the market behaves.

## Risk Parity Portfolio

An alternative approach to Markowitz theory is given by the *risk parity*. A risk parity portfolio is an investment allocation strategy which focuses on the allocation of risk, rather than the allocation of capital. 

A risk parity (equal risk) portfolio is characterised by having equal risk contributions to the total risk from each individual asset. 
Risk parity allocation is also referred to as equally-weighted risk contributions portfolio method. Equally-weighted risk contributions is not about having the same volatility, it is about having each asset contributing in the same way to the portfolio overall volatility. 

For this we will have to define the contribution of each asset to the portfolio risk. **This allocation strategy has gained popularity in the last decades since it is believed to provide better risk adjusted return than capital based allocation strategies.**

Consider a portfolio of $N$ assets: $x_1,\ldots , x_N$ where as usual the weight of the asset $x_i$ is denoted by $w_i$. The $w_i$ form the allocation vector $\mathbf{w}$. Let us further denote the covariance matrix of the assets as $\Sigma$. The volatility of the portfolio is then defined as:

$$\sigma_p={\sqrt {\mathbf{w^T}\Sigma \mathbf{w}}} = \sum _{i=1}^{N}\sigma _{i}\qquad\textrm{with}~\sigma _{i} = w_{i}\cdot \cfrac{\partial\sigma_p}{\partial w_{i}}={\cfrac {w_{i}(\Sigma \mathbf{w})_{i}}{\sqrt {\mathbf{w^T}\Sigma \mathbf{w}}}}$$
so that $\sigma _{i}$ can be interpreted as the contribution of asset $i$ in the portfolio to the overall risk of the portfolio.
Equal risk contribution then means $\sigma _{i} =\sigma _{j}$ for all
$i,j$ or equivalently $\sigma _{i}=\sigma_p/N$. So

$$\sigma _{i} = \cfrac{\sigma_p}{N}={\cfrac {w_{i}(\Sigma \mathbf{w})_{i}}{\sqrt {\mathbf{w^T}\Sigma \mathbf{w}}}}\implies w_{i} = \frac {\sigma_p^{2}}{(\Sigma \mathbf{w})_{i}N}$$

Since we want the previous expression to be true for each $i$, the solution for the weights can be found by solving the minimisation problem

$$\underset{\mathbf{w}}{\min } \sum _{i=1}^{N}\left[w_{i}-{\frac {\sigma_p^{2}}{(\Sigma \mathbf{w})_{i}N}}\right]^{2}$$

In [276]:
num_assets = 5
def risk_parity(w, cov):
    variance = w.T.dot(cov.dot(w)) 
    sum = 0
    N = len(w)
    for i in range(N):
        sum += (w[i] - (variance/(N*cov.dot(w)[i])))**2 
    return sum

args = (covariance,)
constraints = ({'type': 'eq', 'fun': sum_weights}) 
bounds = tuple((0, 1) for asset in range(num_assets))
weights = [1./num_assets for _ in range(num_assets)]
opts = minimize(risk_parity, weights, args=(covariance,),
                bounds=bounds, constraints=constraints)
print (opts)

     fun: 2.2543755363349716e-07
     jac: array([-7.06474804e-04,  1.97343062e-04, -4.26326934e-05, -8.04037852e-06,
        1.11882164e-03])
 message: 'Optimization terminated successfully.'
    nfev: 38
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.25895309, 0.18118756, 0.19670379, 0.22208923, 0.14106633])


In [277]:
sigma_i = []
std = np.sqrt(opts.x.T.dot(covariance.dot(opts.x))) 
for i in range(num_assets):
    a = opts.x[i]*covariance.dot(opts.x)[i] 
    sigma_i.append(a/std)
                   
for i in range(num_assets):
    print ("Risk contribution for asset {}: {:.3f}%".format(i, sigma_i[i]/sum(sigma_i)*100))

Risk contribution for asset 0: 19.974%
Risk contribution for asset 1: 19.999%
Risk contribution for asset 2: 19.990%
Risk contribution for asset 3: 19.993%
Risk contribution for asset 4: 20.044%


### Risk Budget Allocation
The same technique can be used if we would like to calculate a portfolio with risk budget allocation. If we consider the previous equation

$$\sigma _{i}=\cfrac{\sigma_p}{N}$$

where we set the risk contribution fraction to every asset to $1/N$;
now we can simply replace $1/N$ with the desired fraction ($f_i$) for each asset:

$$\sigma _{i}=f_i \cdot \sigma_p$$
so that the relation to minimise becomes:

$$\underset{\mathbf{w}}{\min } \sum _{i=1}^{N}\left[w_{i}-{\frac {f_i \cdot \sigma_p^{2}}{(\Sigma \mathbf{w})_{i}}}\right]^{2} $$

Translating it to $\tt{python}$ we get:

In [278]:
def risk_budget(w, target_risk, cov):
    variance = w.T.dot(cov.dot(w)) 
    sum = 0
    N = len(w)
    for i in range(N):
        sum += (w[i] - (target_risk[i]*variance)/(cov.dot(w)[i]))**2 
    return sum

f_i = [0.3, 0.2, 0.2, 0.15, 0.15] 
args = (f_i, covariance)
constraints = ({'type': 'eq', 'fun': sum_weights})          
bounds = tuple((0, 1) for asset in range(num_assets))
weights = [1./num_assets for _ in range(num_assets)]

opts = minimize(risk_budget, weights, args=(f_i, covariance), 
                bounds=bounds, constraints=constraints)

print (opts)

     fun: 4.798712878867406e-08
     jac: array([-3.11920407e-04,  3.24431247e-04,  9.84173665e-05, -9.10631775e-05,
        4.14667119e-04])
 message: 'Optimization terminated successfully.'
    nfev: 45
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.34604471, 0.17973628, 0.19407208, 0.16911722, 0.11102971])


In [280]:
sigma_i = []
std = np.sqrt(opts.x.T.dot(covariance.dot(opts.x))) 
for i in range(num_assets):
    a = opts.x[i]*covariance.dot(opts.x)[i] 
    sigma_i.append(a/std)
    
for i in range(num_assets):
    print ("Risk contribution for asset {}: {:.3f}%".format(i, sigma_i[i]/sum(sigma_i)*100))

Risk contribution for asset 0: 29.986%
Risk contribution for asset 1: 20.009%
Risk contribution for asset 2: 19.999%
Risk contribution for asset 3: 14.992%
Risk contribution for asset 4: 15.013%


## Portfolio Diversification 

Diversification allows to combine risky stocks so that their combination (the portfolio) is less risky than any of its components. 

Suppose there are two companies located on an isolated island whose chief industry is tourism. 
One company manufactures suntan lotion. Its stock predictably performs well in sunny years and poorly in rainy ones. The other company produces disposable umbrellas. Its stock performs equally poorly in sunny years and well in rainy ones. 

Each company earns a 12% average return. In purchasing either stock, investors incur a great amount of risk because of variability in the stock price driven by fluctuations in weather conditions. Investing half the funds in the suntan lotion stock and half in the stock of the umbrella manufacturer, however, results in a return of 12% regardless of which weather condition prevails. Portfolio diversification thus transforms two
risky stocks, each with an average return of 12%, into a riskless portfolio certain of earning the expected 12%.

Unfortunately, the perfect negative relationship between the returns on these two stocks is very rare in the real world. To some extent, corporate securities move together, so complete elimination of risk through simple portfolio diversification is impossible. However, as long as some lack of parallelism in the returns of securities exists, diversification will always reduce risk.

Empirical studies have demonstrated that unsystematic risk can be virtually eliminated in
portfolios of 30 to 40 randomly selected stocks. Of course, if investments are made in closely related industries, more securities are required to eradicate unsystematic risk.

As we have already seen no measure of unsystematic risk appears in the risk premium, of
CAPM model, since it is assumed that diversification has eliminated it.

In the Markowitz model instead diversification is achieved by seeking to combine in a portfolio assets with returns that are less than perfectly positively correlated, in an effort to lower portfolio risk (variance) without sacrificing return, through the reduction of the correlation matrix $\Sigma$.

### Maximum Diversification Portfolio

Divesrification is most often either pursued in tandem with other objective, such as return maximization, or pursued simply by including more asset classes in out portfolio.

But it does not have to be this way; diversification can be pursued explicitly as the sole objective in portfolio construction.

In a 2008 paper, the diversification ratio $D$ of a portfolio has been defined as:

$$D=\cfrac{\mathbf{w^T}\boldsymbol{\sigma}}{\sqrt {\mathbf{w^T}\Sigma \mathbf{w}}}$$

where $\boldsymbol{\sigma}$ is the vector of volatilities and $\Sigma$ is the covariance matrix. The term in the denominator is the volatility of the portfolio and the term in the numerator is the weighted average volatility of the assets. More diversification within a portfolio decreases the denominator and leads to a higher diversification ratio.

# Machine Learning

In this lesson we will see how machine learning techniques can be successfully applied to solve financial problems. We will first do a quick tour on the theory behind neural networks and then we will see few examples of practical applications regarding regression and classification issues. 

**Disclaimer**: this lecture just scratches the surface of the machine learning topic which has seen a huge development in the latest years leading to thousands of applications in many different fields.

## Neural Network Definition
Artificial Neural Networks (ANN or simply NN) are information processing models that are developed by inspiring from the working principles of human brain. Their most essential property is the ability of learning from sample sets. 

The basic unit of ANN architecture are neurons which are internally in connection with other neurons. 

![Model of an artificial neuron.](neuron.jpeg)

A neuron consists of weights ($w_i$) and real numbers ($x_i$). All inputs injected into a neuron are individually weighted, added together (sometimes it is added also a bias $w_0$) and passed into the activation function which produce the neuron output

$$ \textrm{in} = \sum_{i=1}^{N} x_i w_i +w_0 \rightarrow \textrm{out} = f(\textrm{in})$$  

There are many different types of activation function but one of the simplest is the *step function* which returns just 0 or 1 according to the input value (another is the *sigmoid* which can be thought of as the continuous version of the step function). 

![Sigmoid function.](sigmoid.png)

Other commonly used activation functions are Rectified Linear Unit (ReLU), Tan Hyperbolic (tanh) and Identity function.

For an deeper discussion of activation functions see [this article](https://medium.com/the-theory-of-everything/understanding-activation-functions-in-neural-networks-9491262884e0)

## Training of a neuron

When teaching children how to recognize a bus, we just tell them, showing an example: “This is a bus. That is not a bus.” until they learn the concept of what a bus is. 
Furthermore, if the child sees new objects that she hasn’t seen before, we could expect her to recognize correctly whether the new object is a bus or not.

This is exactly the idea behind neurons.
Similarly, inputs from a *training* set are presented to the neuron one after the other together with the correct output and the neuron weights are modified accordingly.

When an entire pass through all of the input training vectors is completed (*epoch*) the neuron has learnt ! 

At this time, if an input vector $\mathbf{x}$ (already in the training set) is given to the neuron, it will output the correct value. If $\mathbf{x}$ is not in the training set, the network will respond with an output similar to other training vectors close to $\mathbf{x}$.

This kind of training is called *supervised*. It’s supervised because we have a set of training data with known targets and, during training, we want our model to learn to predict the target from the other variables. 

Unfortunately using just a neuron is not too useful since it is not possible to solve
the interesting problems we would like to face with just that simple architecture. The next step is then to put together more neurons in *layers*.

### Multilayered Neural networks

![A multilayered neural network.](multilayer.jpeg)

In a multilayered NN each neuron from the *input layer* is fed up to each neuron in the next hidden layer, and from there to each neuron on the output layer. We should note that there can be any number of neurons per layer and there are usually multiple hidden layers to pass through before ultimately reaching the output layer.

### Training a Multilayered Neural Network

The training of a multilayered NN follows these steps:

* present a training sample to the neural network (initialized with random weights $w_i$);
* compute the network output obtained by calculating activations of each layer;
* calculate the error (loss) as the difference between the NN predicted output and the target output;
* having calculated the error, re-adjust the weights of the network such that the difference with the target decreases;
* continue the process for all samples several times (epochs) until the weights are not changing too much (i.e. the process converged).

<img src="training_nn.png">

The NN error is computed by the *loss function*. Different loss functions will give different errors for the same prediction, and thus have a considerable effect on the performance of the model. Two are the main possible choices

* Mean Absolute Error (MAE): the average of the absolute value of the differences between the predictions and true values;
* Root Mean Squared Error (MSE): the square root of the average of the squared differences between the predictions and true values.

The mean absolute error is easily interpretable, as it represents how far off we are on average from the correct value. The root mean squared error penalizes larger errors more heavily and is commonly used in regression tasks. Either metrics may be appropriate depending on the situation and you can use both for comparison. [Here](https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d) is a discussion of the merits of these metrics.

The error or loss is a function of the internal parameters of the model (i.e the weights and bias). For accurate predictions, one needs to minimize the calculated error.
In a neural network, this is done using *back propagation* (see [this article](https://towardsdatascience.com/understanding-backpropagation-algorithm-7bb3aa2f95fd)). The current error is typically propagated backwards to a previous layer, where it is used to modify the weights and bias in such a way that the error is minimized.

<img src="loss_function.png">

The weights are modified using a function called Optimization Function (we will use *Adam* as optimizator in the following but there are more).

### Neural Network Design

There is no rule to guide developer into the design of a neural network in terms of number of layers and neuron per layer. The most common strategy is a trail and error one where you finally pick up the solution giving the best accuracy. In general a larger number of nodes is better to catch highly structured data with a lot of feature although it may require larger training sample to work correctly.

A common mistake to avoid is to *overtrain* a NN. Overtraining is what happens when the NN learns too well the training sample but its performance degrade substantially in an independent testing sample. 

So usually it is required to split the available sample in two parts: training and testing (e.g. 80% and 20%) and to use the former to perform the training and the latter to cross-check the performance. **Usually performance are measured using the loss function value at the end of the training.**

Anyway as a rule of thumb a NN with just one hidden layer with a number of neurons averaging the inputs and outputs is sufficient in most cases. In the following we will use more complex networks just for illustration, no strong attempt in optimizing the layout has been done though.

## Regression and Classification
The two main categories of problems that can be solved with neural networks are *classification* and *regression*. Let's see their characteristics and differences.

### Classification 
Classification is a process of finding a function which helps in dividing the dataset into classes based on different parameters. 

The task of the classification algorithm is to find the mapping function to map the input($x$) to the **discrete** output($y$).
We try to find the decision boundary, which can divide the dataset into different classes.

Example: the best example to understand the classification problem is email spam detection. The model is trained on the basis of millions of emails on different parameters, and whenever it receives a new email, it identifies whether the email is spam or not.
Classification algorithms can also be in speech recognition, car plates identification, etc.

### Regression
Regression is a process of finding the correlations between dependent and independent variables. It helps in predicting the continuous variables such as prediction of market trends, prediction of house prices, etc.

The task of the regression algorithm is to find the mapping function to map the input variable($x$) to the **continuous** output variable($y$).
We try to find the best fit line, which can predict the output more accurately. 	

Example: suppose we want to do weather forecasting, so for this, we will use a regression algorithm. In weather prediction, the model is trained on the past data, and once the training is completed, it can predict the weather for future days.
In general whenever we are dealing with function approximation this kind of algorithms can be applied. 	

### Technical Note
Neural network training and testing is performed using two modules: $\tt{keras}$ (which in turn is based on a Google opensource library called $\tt{tensorflow}$) and $\tt{scikit-learn}$ which provide many useful utilitites for the training.

In order hide as much as possible the many little details that have to be set when developing NN I have developed a simple class ($\tt{FinNN}$) which relies on $\tt{keras}$ anyway but should make the whole process easier.

## Function approximation 

As a first practical example let's try to design an ANN which is capable of learning the functional form underlying a set of data.

Let's generate a sample with $x$ (input), $f(x)$ (target output) pairs where $f(x) = x^3 +2$ and let's start to code the NN structure. 

We start by importing the necessary modules.

In [19]:
from finnn import FinNN
import numpy as np

Then we generate the training sample (i.e. the $x$, $f(x)$ pairs) and apply a simple transformation on the sample in order to have all the inputs and outputs in the $[0, 1]$ range. This is usually done to provide the NN with *normalized* data, infact the NN can be fooled by large or very small numbers giving unstable results.

In [20]:
# define the dataset
x = np.array([i for i in np.arange(-2, 2, 0.001)])

y = np.array([i**3+2 for i in x])
print("Distribution of original data ", x.min(), x.max(), y.min(), y.max())

trainer = FinNN("ANN")
trainer.setData(x, y, test_size=0.2)
trainer.normalize()

# here you should see that x and y are between 0 and 1
print("The same data after the normalization ", trainer.x.min(), 
      trainer.x.max(), trainer.y.min(), trainer.y.max())

Distribution of original data  -2.0 1.9989999999995596 -6.0 9.98800599899472
The same data after the normalization  0.0 1.0 0.0 0.9999999999999999


Next we can define the structure of the neural network. There is no predefined rule to decide the number of layers and nodes you need to go by trial and error. Here the problem is quite simple so there is no need to use a complecated NN. 

In the end I have decided to use two layers with 15 and 5 neurons and a *tanh* activation function. The $\tt{inputs}$ parameter has to be set to 1 since we have just one single input, the $x$ value. 

In [21]:
# design the neural network model
trainer.addInputLayer(inputs=1, neurons=15, activation='tanh')
trainer.addHiddenLayer(neurons=5, activation='tanh')
trainer.addOutputLayer(outputs=1)

# define the loss function (mean squared error) 
# and optimization algorithm (Adam)
trainer.compileModel(loss='mse', opt='adam')

# fit the model on the training dataset
# using 500 epochs, a batch_size of 10
trainer.fit(epochs=2000, verbose=1)

480
Epoch 1/2000
3200/3200 [==============================] - 0s 31us/step - loss: 0.2362
Epoch 2/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.1498
Epoch 3/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.0845
Epoch 4/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.0453
Epoch 5/2000
3200/3200 [==============================] - 0s 8us/step - loss: 0.0263
Epoch 6/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.0196
Epoch 7/2000
3200/3200 [==============================] - 0s 7us/step - loss: 0.0182
Epoch 8/2000
3200/3200 [==============================] - 0s 7us/step - loss: 0.0175
Epoch 9/2000
3200/3200 [==============================] - 0s 7us/step - loss: 0.0164
Epoch 10/2000
3200/3200 [==============================] - 0s 7us/step - loss: 0.0152
Epoch 11/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.0141
Epoch 12/2000
3200/3200 [==============================] -

3200/3200 [==============================] - 0s 5us/step - loss: 0.0062
Epoch 97/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0062
Epoch 98/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0061
Epoch 99/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0061
Epoch 100/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0061
Epoch 101/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.0060
Epoch 102/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0060
Epoch 103/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0060
Epoch 104/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0059
Epoch 105/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0059
Epoch 106/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0058
Epoch 107/2000
3200/3200 [==============================] - 0

3200/3200 [==============================] - 0s 5us/step - loss: 0.0033
Epoch 191/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0033
Epoch 192/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.0033
Epoch 193/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0033
Epoch 194/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0032
Epoch 195/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0032
Epoch 196/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0032
Epoch 197/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0031
Epoch 198/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.0031
Epoch 199/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0031
Epoch 200/2000
3200/3200 [==============================] - 0s 6us/step - loss: 0.0031
Epoch 201/2000
3200/3200 [==============================] 

3200/3200 [==============================] - 0s 4us/step - loss: 0.0011
Epoch 285/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0011
Epoch 286/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0011
Epoch 287/2000
3200/3200 [==============================] - 0s 3us/step - loss: 0.0011
Epoch 288/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0011
Epoch 289/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0011
Epoch 290/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0010
Epoch 291/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0010
Epoch 292/2000
3200/3200 [==============================] - 0s 4us/step - loss: 0.0010
Epoch 293/2000
3200/3200 [==============================] - 0s 5us/step - loss: 0.0010
Epoch 294/2000
3200/3200 [==============================] - 0s 4us/step - loss: 9.9128e-04
Epoch 295/2000
3200/3200 [============================

3200/3200 [==============================] - 0s 4us/step - loss: 3.4889e-04
Epoch 375/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.4444e-04
Epoch 376/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.4007e-04
Epoch 377/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.3508e-04
Epoch 378/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.3037e-04
Epoch 379/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.2690e-04
Epoch 380/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.2115e-04
Epoch 381/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.1830e-04
Epoch 382/2000
3200/3200 [==============================] - 0s 6us/step - loss: 3.1344e-04
Epoch 383/2000
3200/3200 [==============================] - 0s 4us/step - loss: 3.0834e-04
Epoch 384/2000
3200/3200 [==============================] - 0s 3us/step - loss: 3.0469e-04
Epoch 385/2000

...

3200/3200 [==============================] - 0s 5us/step - loss: 3.4065e-06
Epoch 1618/2000
3200/3200 [==============================] - 0s 4us/step - loss: 3.1918e-06
Epoch 1619/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.0919e-06
Epoch 1620/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.0556e-06
Epoch 1621/2000
3200/3200 [==============================] - 0s 4us/step - loss: 3.0391e-06
Epoch 1622/2000
3200/3200 [==============================] - 0s 4us/step - loss: 3.2784e-06
Epoch 1623/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.9183e-06
Epoch 1624/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.9824e-06
Epoch 1625/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.9426e-06
Epoch 1626/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.0675e-06
Epoch 1627/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.1617e-06
Epoc

3200/3200 [==============================] - 0s 6us/step - loss: 2.8291e-06
Epoch 1706/2000
3200/3200 [==============================] - ETA: 0s - loss: 2.7295e-0 - 0s 4us/step - loss: 2.8105e-06
Epoch 1707/2000
3200/3200 [==============================] - 0s 4us/step - loss: 3.3262e-06
Epoch 1708/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.3796e-06
Epoch 1709/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.9426e-06
Epoch 1710/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.8700e-06
Epoch 1711/2000
3200/3200 [==============================] - 0s 6us/step - loss: 3.0249e-06
Epoch 1712/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.8065e-06
Epoch 1713/2000
3200/3200 [==============================] - 0s 6us/step - loss: 2.8316e-06
Epoch 1714/2000
3200/3200 [==============================] - 0s 6us/step - loss: 2.9256e-06
Epoch 1715/2000
3200/3200 [==============================] - 0s 6us/

3200/3200 [==============================] - 0s 7us/step - loss: 3.0383e-06
Epoch 1794/2000
3200/3200 [==============================] - 0s 4us/step - loss: 3.5493e-06
Epoch 1795/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.3591e-06
Epoch 1796/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.0333e-06
Epoch 1797/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.9303e-06
Epoch 1798/2000
3200/3200 [==============================] - 0s 5us/step - loss: 3.0027e-06
Epoch 1799/2000
3200/3200 [==============================] - 0s 4us/step - loss: 3.0551e-06
Epoch 1800/2000
3200/3200 [==============================] - 0s 6us/step - loss: 2.6074e-06
Epoch 1801/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.6737e-06
Epoch 1802/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.6934e-06
Epoch 1803/2000
3200/3200 [==============================] - 0s 7us/step - loss: 2.7872e-06
Epoc

3200/3200 [==============================] - 0s 5us/step - loss: 2.6431e-06
Epoch 1883/2000
3200/3200 [==============================] - 0s 6us/step - loss: 2.6252e-06
Epoch 1884/2000
3200/3200 [==============================] - 0s 6us/step - loss: 2.7074e-06
Epoch 1885/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.5999e-06
Epoch 1886/2000
3200/3200 [==============================] - 0s 8us/step - loss: 2.4744e-06
Epoch 1887/2000
3200/3200 [==============================] - 0s 7us/step - loss: 2.4904e-06
Epoch 1888/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.5422e-06
Epoch 1889/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.5685e-06
Epoch 1890/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.5890e-06
Epoch 1891/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.7565e-06
Epoch 1892/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.7339e-06
Epoc

3200/3200 [==============================] - 0s 9us/step - loss: 2.4458e-06
Epoch 1971/2000
3200/3200 [==============================] - 0s 3us/step - loss: 2.3842e-06
Epoch 1972/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.3987e-06
Epoch 1973/2000
3200/3200 [==============================] - 0s 3us/step - loss: 2.5923e-06
Epoch 1974/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.9807e-06
Epoch 1975/2000
3200/3200 [==============================] - 0s 5us/step - loss: 2.7012e-06
Epoch 1976/2000
3200/3200 [==============================] - 0s 3us/step - loss: 3.1535e-06
Epoch 1977/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.7055e-06
Epoch 1978/2000
3200/3200 [==============================] - 0s 3us/step - loss: 2.5651e-06
Epoch 1979/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.4751e-06
Epoch 1980/2000
3200/3200 [==============================] - 0s 4us/step - loss: 2.3710e-06
Epoc

After the training is completed we can evaluate how good it is. To do this we can compute the residuals or the square root of the sum of the squared difference between the true value and the one predicted by the NN. We will also plot the true function and the predicted one in order to have a graphical representation of the goodness of our training.
To have a numerical estimate of the agreement it has been computed also the *mean squared error* defined as:

$$\textrm{MSE} = \cfrac{\sum_{i=1}^n{\big(\frac{x_{i}^{pred} - x_i^{truth}}{x_i^{truth}}\big)^2}}{n}$$

A *perfect* prediction would lead to $\textrm{MSE}=0$ so the lower this number the better the agreement. 

In [22]:
from sklearn.metrics import mean_squared_error

trainer.fullPrediction()
# report model error computing the mean squared error
print('MSE: %.7f' % mean_squared_error(trainer.y, trainer.predictions))

MSE: 0.0000023


To get an idea of what it is going on in the picture below are shown the actual function we want to approximate and different predictions of our NN obtained with four epoch numbers (5, 100, 800, 5000).

<img src="training_vs_epoch.png">

It is clear how the agreement improves with higher number of epochs which means that the NN has more opportunities to adapt the weights and reduce the loss (or error or distance) to the target values. Even in the case of 5000 epochs zooming in you could see discrepancies not visible at the scale of the plot. Remember that increasing too much the number of epochs may lead to overfitting. So in this case if we need more accuracy we need to either increase the training sample or to change the NN design.

To check if this is the case we can *evaluate* our NN with both the training ad the testing samples. If the losses are comparable the NN is ok otherwise if the training losses are much smaller than the testing we had overfitting.

In [23]:
trainer.evaluate()

3200/3200 [==============================] - 0s 25us/step
Training: 2.2547546473106196e-06
800/800 [==============================] - 0s 22us/step
Test: 2.3776677153364288e-06


Since the two numbers are in good agreement we can be confident that our NN didn't overfit.


### Black-Scholes Call Options

The first financial application of a NN concerns the pricing of european call options: essentially we will create a neural network capable of approximate the famous Black-Scholes pricing formula

$$ P_\textrm{call} = F_\textrm{BS}(K, r, \sigma, \mathrm{ttm})$$

Like before we are going to generate the training sample this time made of a grid of volatility-rate pairs $(\sigma, r)$ (for simplicity we are going to set moneyness and time to maturity to 1). The target values are the price of a call computed using the pricing function in the $\tt{finmarkets.py}$ library with the corresponding inputs.

In [26]:
from finmarkets import call

data = []
rates = np.arange(0.01, 0.11, 0.001)
sigmas = np.arange(0.1, 0.6, 0.005)

for r in rates:
    for sigma in sigmas:
        call_price = call(1, r, sigma, 1)
        data.append([r, sigma, call_price]) 

Since it takes some time to generate data samples, it is always advisable to save them in a file since we may need to load it many times during the NN development.
This can be done with $\tt{pandas}$.

In [27]:
import pandas as pd

df = pd.DataFrame()

data = np.array(data)
df['rate'] = data[:, 0]
df['vol'] = data[:, 1]
df['price'] = data[:, 2]

df.to_csv("bs_training_sample.csv")

In [28]:
print (df.describe())

               rate           vol         price
count  10000.000000  10000.000000  10000.000000
mean       0.059500      0.347500      0.165274
std        0.028868      0.144338      0.055387
min        0.010000      0.100000      0.044852
25%        0.034750      0.223750      0.119389
50%        0.059500      0.347500      0.165476
75%        0.084250      0.471250      0.212097
max        0.109000      0.595000      0.277071


Following the previous example we will use the $\tt{FinNN}$ utility class to develop the NN and also we will *normalize* data to get better results.
**Beware that this time we have TWO input parameters (rate and volatilty)** and not just one.

In [29]:
data =  pd.read_csv("bs_training_sample.csv")

x = data.iloc[:, 1:3].values
y = data.iloc[:, 3].values

In [30]:
trainer = FinNN("ANN")
trainer.setData(x, y, test_size=0.2)
trainer.normalize()

# define the NN architecture
trainer.addInputLayer(inputs=2, neurons=20, activation='relu')
trainer.addHiddenLayer(neurons=8, activation='relu')
trainer.addOutputLayer(outputs=1)

trainer.compileModel(loss='mse', opt='adam')
    
trainer.fit(epochs=3000, verbose=1)

1200
Epoch 1/3000
8000/8000 [==============================] - 0s 14us/step - loss: 0.3102
Epoch 2/3000
8000/8000 [==============================] - 0s 2us/step - loss: 0.2756
Epoch 3/3000
8000/8000 [==============================] - 0s 4us/step - loss: 0.2424
Epoch 4/3000
8000/8000 [==============================] - 0s 4us/step - loss: 0.2165
Epoch 5/3000
8000/8000 [==============================] - 0s 3us/step - loss: 0.1900
Epoch 6/3000
8000/8000 [==============================] - 0s 3us/step - loss: 0.1622
Epoch 7/3000
8000/8000 [==============================] - 0s 3us/step - loss: 0.1330
Epoch 8/3000
8000/8000 [==============================] - 0s 4us/step - loss: 0.1035
Epoch 9/3000
8000/8000 [==============================] - 0s 3us/step - loss: 0.0735
Epoch 10/3000
8000/8000 [==============================] - 0s 3us/step - loss: 0.0428
Epoch 11/3000
8000/8000 [==============================] - 0s 3us/step - loss: 0.0220
Epoch 12/3000
8000/8000 [==============================] 

8000/8000 [==============================] - 0s 2us/step - loss: 4.3596e-05
Epoch 94/3000
8000/8000 [==============================] - 0s 3us/step - loss: 4.3127e-05
Epoch 95/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.2782e-05
Epoch 96/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.2427e-05
Epoch 97/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.2059e-05
Epoch 98/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.1671e-05
Epoch 99/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.1330e-05
Epoch 100/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.1071e-05
Epoch 101/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.0840e-05
Epoch 102/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.0362e-05
Epoch 103/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.9974e-05
Epoch 104/3000
8000/

8000/8000 [==============================] - 0s 3us/step - loss: 1.5189e-05
Epoch 184/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.4949e-05
Epoch 185/3000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4775e-05
Epoch 186/3000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4500e-05
Epoch 187/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.4165e-05
Epoch 188/3000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4084e-05
Epoch 189/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.3812e-05
Epoch 190/3000
8000/8000 [==============================] - 0s 3us/step - loss: 1.3564e-05
Epoch 191/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.3549e-05
Epoch 192/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.3289e-05
Epoch 193/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.2883e-05
Epoch 194/3000

8000/8000 [==============================] - 0s 2us/step - loss: 5.5688e-06
Epoch 273/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.6004e-06
Epoch 274/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.5828e-06
Epoch 275/3000
8000/8000 [==============================] - 0s 3us/step - loss: 5.4670e-06
Epoch 276/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.4688e-06
Epoch 277/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.3981e-06
Epoch 278/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.4031e-06
Epoch 279/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.4282e-06
Epoch 280/3000
8000/8000 [==============================] - 0s 3us/step - loss: 5.3136e-06
Epoch 281/3000
8000/8000 [==============================] - 0s 3us/step - loss: 5.2762e-06
Epoch 282/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.2461e-06
Epoch 283/3000

8000/8000 [==============================] - 0s 2us/step - loss: 3.7145e-06
Epoch 362/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.7444e-06
Epoch 363/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.7067e-06
Epoch 364/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.6522e-06
Epoch 365/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.6224e-06
Epoch 366/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.5869e-06
Epoch 367/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.5585e-06
Epoch 368/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.5168e-06
Epoch 369/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.5532e-06
Epoch 370/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.5024e-06
Epoch 371/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.5220e-06
Epoch 372/3000

...

8000/8000 [==============================] - 0s 2us/step - loss: 3.7418e-07
Epoch 2583/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.7550e-07
Epoch 2584/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.2190e-07
Epoch 2585/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.8506e-07
Epoch 2586/3000
8000/8000 [==============================] - 0s 2us/step - loss: 6.8412e-07
Epoch 2587/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.5980e-06
Epoch 2588/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.2970e-07
Epoch 2589/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.4202e-07
Epoch 2590/3000
8000/8000 [==============================] - 0s 3us/step - loss: 3.7676e-07
Epoch 2591/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.8864e-07
Epoch 2592/3000
8000/8000 [==============================] - 0s 2us/step - loss: 7.8230e-07
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 8.3556e-07
Epoch 2671/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.6406e-07
Epoch 2672/3000
8000/8000 [==============================] - 0s 3us/step - loss: 4.7599e-07
Epoch 2673/3000
8000/8000 [==============================] - 0s 2us/step - loss: 6.2303e-07
Epoch 2674/3000
8000/8000 [==============================] - 0s 3us/step - loss: 5.5064e-07
Epoch 2675/3000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2379e-06
Epoch 2676/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.3717e-06
Epoch 2677/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.0259e-06
Epoch 2678/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.8446e-07
Epoch 2679/3000
8000/8000 [==============================] - 0s 2us/step - loss: 7.1950e-07
Epoch 2680/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.2789e-07
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 4.4353e-07
Epoch 2759/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.8028e-07
Epoch 2760/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.3877e-07
Epoch 2761/3000
8000/8000 [==============================] - 0s 2us/step - loss: 6.6664e-07
Epoch 2762/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.7180e-06
Epoch 2763/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.0736e-06
Epoch 2764/3000
8000/8000 [==============================] - 0s 2us/step - loss: 8.2793e-07
Epoch 2765/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.0675e-06
Epoch 2766/3000
8000/8000 [==============================] - 0s 2us/step - loss: 7.6133e-07
Epoch 2767/3000
8000/8000 [==============================] - 0s 2us/step - loss: 8.0896e-07
Epoch 2768/3000
8000/8000 [==============================] - 0s 2us/step - loss: 6.6344e-07
Epoc

8000/8000 [==============================] - 0s 2us/step - loss: 8.6275e-07
Epoch 2848/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.6500e-07
Epoch 2849/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.5828e-07
Epoch 2850/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.8386e-07
Epoch 2851/3000
8000/8000 [==============================] - 0s 2us/step - loss: 8.1136e-07
Epoch 2852/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.4008e-07
Epoch 2853/3000
8000/8000 [==============================] - 0s 2us/step - loss: 9.1259e-07
Epoch 2854/3000
8000/8000 [==============================] - 0s 2us/step - loss: 3.6509e-06
Epoch 2855/3000
8000/8000 [==============================] - 0s 2us/step - loss: 8.1723e-07
Epoch 2856/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.0893e-06
Epoch 2857/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.4082e-07
Epoc

8000/8000 [==============================] - 0s 2us/step - loss: 5.1851e-07
Epoch 2936/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.1479e-06
Epoch 2937/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.5473e-06
Epoch 2938/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.1321e-06
Epoch 2939/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.8377e-07
Epoch 2940/3000
8000/8000 [==============================] - 0s 2us/step - loss: 4.6448e-07
Epoch 2941/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.1927e-07
Epoch 2942/3000
8000/8000 [==============================] - 0s 2us/step - loss: 8.2158e-07
Epoch 2943/3000
8000/8000 [==============================] - 0s 2us/step - loss: 5.6954e-07
Epoch 2944/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.0307e-06
Epoch 2945/3000
8000/8000 [==============================] - 0s 2us/step - loss: 1.1983e-06
Epoc

In [31]:
trainer.evaluate()

# when the training takes some time it is useful
# to save the model weights in a file to use it later on
trainer.saveModel('black_scholes')

8000/8000 [==============================] - 0s 20us/step
Training: 6.939637916048014e-07
2000/2000 [==============================] - 0s 21us/step
Test: 6.693533414363628e-07


In [32]:
np.sqrt(7.e-7)

0.0008366600265340755

As you can see the training and test samples give roughly the same MSE value so we are reasonably sure that there hasn't been *overfitting*.

Again we can evaluate graphically how good it is.
<img src="vol_rate.png">

In general to judge if the level of accuracy we have reached is enough you have to

* if you are using the metric MSE you need to make the $\sqrt{\mathrm{MSE}}$ to get the *real* error
* apply this error to a typical output and check if the accuracy is enough.

In this example we know that the our prices go from 0.04 to 0.28 and the final accuracy is 0.0008. But since we are working with the moneyness we need to multiply those values for a typical strike, say 100. So in the worst case we know are able to price our call as $40 \pm 0.08$, which is not bad for our study but may not be ideal for deciding if we would like to invest in this call or not. 

We can also compare the prediction in a practical case; let's say we want to know the price of a call (with moneyness 1 and time to maturity 1 year) when the interest rate is 0.015 and the volatility 0.234:

In [35]:
import numpy as np
from finmarkets import call

# here we load the trained model
trainer.loadModel('black_scholes')

# this is our input vector
rv = np.array([[0.015, 0.234]])
# here we compare the predection with the BS call price                 
print ('{} => {:.4f} (expected {:.4f})'.format(rv.tolist(), 
                                        trainer.predict(rv)[0][0], 
                                        call(1, rv[0][0], rv[0][1], 1)))

[[0.015, 0.234]] => 0.1001 (expected 0.1001)


It is very import to remeber that a **NN cannot extrapolate**. Indeed if you try to predict the price of a call from rate and volatility outside the training *phase space* (with values that aren't in the intervals used in the training), say $r = 0.22$ and $\sigma = 0.01$...

In [37]:
# this is our input vector
rv = np.array([[0.22, 0.01]])
                 
# here we compare the predection with the BS call price                 
print ('{} => {:.4f} (expected {:.4f})'.format(rv.tolist(), 
                                        trainer.predict(rv)[0][0], 
                                        call(1, rv[0][0], rv[0][1], 1)))

[[0.22, 0.01]] => 0.1595 (expected 0.1975)


## Model Calibration

The function approximation capabilities of a neural network can serve other scopes rather than predicting the function values. A very useful application is indeed *model calibration* which consists of deriving parameters of a model directly from market values. This is especially convenient to estimate parameters (e.g. volatility) which are otherwise complicated to compute.

Assume we need to estimate the *implied volatilty* of a stock price in real time. If in the market are available call options with our stock as underlying we can exploit again the Black and Scholes formula. The idea is in fact to train a NN where the input is a list of price, moneyness, rate and time to maturity $(P_\textrm{call}, K, r, \mathrm{ttm})$ and the target output is the volatility derived from the inversion of the call option pricing formula

$$ \sigma = F^{-1}_\textrm{BS}(P_\textrm{call}, K, r, \mathrm{ttm})$$

We can than calibrate our model by predicting the stock volatility with the trained NN using as input the market price of the option and its characteristics.

### Historical vs. Implied Volatility

Historical volatility is the realized volatility of the underlying asset over a previous time period. It is determined by measuring the standard deviation of the underlying asset from the mean during that time period.
This estimate differs from the Black-Scholes method's implied volatility, as it is based on the **actual** volatility of the underlying asset. 

Using historical volatility has some drawbacks. Volatility shifts as markets go through different regimes. Thus, historical volatility may not be an accurate measure of future volatility. Implied volatility takes into account all of the information used by market participants to determine prices in the options market, instead of just past prices.

As an example we can reuse the training sample created before (again we are going to set $\mathrm{ttm}=1$ and $K=1$). Clearly now $\tt{x}$ will be pairs of rate and price and $\tt{y}$ the volatility.

In [58]:
import pandas as pd
from finnn import FinNN

data =  pd.read_csv("bs_training_sample.csv")

# concatenate price and rate
df = pd.concat([data.iloc[:, 1:2], data.iloc[:, 3:4]], 1)

x = df.values
y = data.iloc[:, 2].values

In [59]:
trainer = FinNN("ANN")

trainer.setData(x, y, test_size=0.20)
trainer.normalize()

trainer.addInputLayer(inputs=2, neurons=20, activation='relu')
trainer.addHiddenLayer(neurons=8, activation='relu')
trainer.addOutputLayer(outputs=1)

trainer.compileModel(loss='mse', opt='adam')
trainer.fit(epochs=2000, verbose=1)

trainer.evaluate()
trainer.saveModel("calibration")

1200
Epoch 1/2000
8000/8000 [==============================] - 0s 14us/step - loss: 0.3347
Epoch 2/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.3347
Epoch 3/2000
8000/8000 [==============================] - 0s 3us/step - loss: 0.3347
Epoch 4/2000
8000/8000 [==============================] - 0s 2us/step - loss: 0.3347
Epoch 5/2000
8000/8000 [==============================] - 0s 2us/step - loss: 0.3347
Epoch 6/2000
8000/8000 [==============================] - 0s 2us/step - loss: 0.3346
Epoch 7/2000
8000/8000 [==============================] - 0s 2us/step - loss: 0.3342
Epoch 8/2000
8000/8000 [==============================] - 0s 3us/step - loss: 0.3328
Epoch 9/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.3264
Epoch 10/2000
8000/8000 [==============================] - 0s 2us/step - loss: 0.3117
Epoch 11/2000
8000/8000 [==============================] - 0s 3us/step - loss: 0.2780
Epoch 12/2000
8000/8000 [==============================] 

8000/8000 [==============================] - 0s 2us/step - loss: 2.2616e-04
Epoch 95/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2012e-04
Epoch 96/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1420e-04
Epoch 97/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.0871e-04
Epoch 98/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.0337e-04
Epoch 99/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.9854e-04
Epoch 100/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.9392e-04
Epoch 101/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.8929e-04
Epoch 102/2000
8000/8000 [==============================] - 0s 2us/step - loss: 1.8497e-04
Epoch 103/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.8089e-04
Epoch 104/2000
8000/8000 [==============================] - 0s 2us/step - loss: 1.7695e-04
Epoch 105/2000
8000

8000/8000 [==============================] - 0s 3us/step - loss: 8.5277e-05
Epoch 185/2000
8000/8000 [==============================] - 0s 2us/step - loss: 8.4979e-05
Epoch 186/2000
8000/8000 [==============================] - 0s 2us/step - loss: 8.4774e-05
Epoch 187/2000
8000/8000 [==============================] - 0s 2us/step - loss: 8.4535e-05
Epoch 188/2000
8000/8000 [==============================] - 0s 2us/step - loss: 8.4285e-05
Epoch 189/2000
8000/8000 [==============================] - 0s 2us/step - loss: 8.4139e-05
Epoch 190/2000
8000/8000 [==============================] - 0s 3us/step - loss: 8.3877e-05
Epoch 191/2000
8000/8000 [==============================] - 0s 2us/step - loss: 8.3668e-05
Epoch 192/2000
8000/8000 [==============================] - 0s 3us/step - loss: 8.3420e-05
Epoch 193/2000
8000/8000 [==============================] - 0s 2us/step - loss: 8.3239e-05
Epoch 194/2000
8000/8000 [==============================] - 0s 2us/step - loss: 8.3016e-05
Epoch 195/2000

8000/8000 [==============================] - 0s 3us/step - loss: 7.1985e-05
Epoch 274/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.1842e-05
Epoch 275/2000
8000/8000 [==============================] - 0s 2us/step - loss: 7.1726e-05
Epoch 276/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.1595e-05
Epoch 277/2000
8000/8000 [==============================] - 0s 2us/step - loss: 7.1515e-05
Epoch 278/2000
8000/8000 [==============================] - 0s 2us/step - loss: 7.1389e-05
Epoch 279/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.1385e-05
Epoch 280/2000
8000/8000 [==============================] - 0s 2us/step - loss: 7.1231e-05
Epoch 281/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.1104e-05
Epoch 282/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.1061e-05
Epoch 283/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.0861e-05
Epoch 284/2000

8000/8000 [==============================] - 0s 2us/step - loss: 6.2776e-05
Epoch 364/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.2677e-05
Epoch 365/2000
8000/8000 [==============================] - ETA: 0s - loss: 6.0229e-0 - 0s 3us/step - loss: 6.2549e-05
Epoch 366/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.2475e-05
Epoch 367/2000
8000/8000 [==============================] - 0s 2us/step - loss: 6.2304e-05
Epoch 368/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.2216e-05
Epoch 369/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.2231e-05
Epoch 370/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.2282e-05
Epoch 371/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.2077e-05
Epoch 372/2000
8000/8000 [==============================] - 0s 2us/step - loss: 6.1829e-05
Epoch 373/2000
8000/8000 [==============================] - 0s 3us/step - los

...

8000/8000 [==============================] - 0s 2us/step - loss: 2.6642e-06
Epoch 1609/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.5599e-06
Epoch 1610/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.5680e-06
Epoch 1611/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.4811e-06
Epoch 1612/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.4578e-06
Epoch 1613/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.4014e-06
Epoch 1614/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.4971e-06
Epoch 1615/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.5082e-06
Epoch 1616/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.5909e-06
Epoch 1617/2000
8000/8000 [==============================] - 0s 5us/step - loss: 2.3759e-06
Epoch 1618/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.3903e-06
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 2.2030e-06
Epoch 1697/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.3588e-06
Epoch 1698/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.5760e-06
Epoch 1699/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.3587e-06
Epoch 1700/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.4656e-06
Epoch 1701/2000
8000/8000 [==============================] - ETA: 0s - loss: 2.0599e-0 - 0s 4us/step - loss: 2.5237e-06
Epoch 1702/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.9578e-06
Epoch 1703/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.1340e-06
Epoch 1704/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.7894e-06
Epoch 1705/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.5306e-06
Epoch 1706/2000
8000/8000 [==============================] - 0s 3us/

8000/8000 [==============================] - 0s 2us/step - loss: 2.0883e-06
Epoch 1785/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1187e-06
Epoch 1786/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.2580e-06
Epoch 1787/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.1941e-06
Epoch 1788/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1838e-06
Epoch 1789/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2436e-06
Epoch 1790/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.3408e-06
Epoch 1791/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.4837e-06
Epoch 1792/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2907e-06
Epoch 1793/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.2207e-06
Epoch 1794/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.1222e-06
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 2.3008e-06
Epoch 1873/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1730e-06
Epoch 1874/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.3273e-06
Epoch 1875/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.2164e-06
Epoch 1876/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.4108e-06
Epoch 1877/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.5794e-06
Epoch 1878/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.9434e-06
Epoch 1879/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.9276e-06
Epoch 1880/2000
8000/8000 [==============================] - 0s 6us/step - loss: 2.7907e-06
Epoch 1881/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2813e-06
Epoch 1882/2000
8000/8000 [==============================] - 0s 2us/step - loss: 2.2188e-06
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 2.2087e-06
Epoch 1962/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1041e-06
Epoch 1963/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.3346e-06
Epoch 1964/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1120e-06
Epoch 1965/2000
8000/8000 [==============================] - 0s 2us/step - loss: 1.9494e-06
Epoch 1966/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.9901e-06
Epoch 1967/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1585e-06
Epoch 1968/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2092e-06
Epoch 1969/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1996e-06
Epoch 1970/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2864e-06
Epoch 1971/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.0471e-06
Epoc

Provided our training includes the correct range of market prices of our call we can quickly and easily estimate the implied volatility. For example if the risk-free rate is 2% and the current price is 0.15 (remember that we are using the BS formula in terms of moneyness)

In [60]:
trainer.loadModel('calibration')
rv = np.array([[0.02, 0.15]])

print ('{} => {:.4f} (expected call price {:.4f})'.format(rv.tolist(), 
                                        trainer.predict(rv)[0][0], 
                                        call(1, 0.02, trainer.predict(rv)[0][0], 1)))

[[0.02, 0.15]] => 0.3559 (expected call price 0.1500)



## Neural Network to Recognize Handwritten Digits

We don't usually appreciate how tough a problem our visual system solve, maybe it is enough to consider that it involves 5 visual cortices containing 140 million neurons each. 
Anyway the difficulties of visual pattern recognition become apparent if you attempt to write a computer program to recognize digits like those below

<img src="mnist_100_digits.png">

Simple intuition about how we recognize shapes (e.g. a 9 has a loop at the top, and a vertical stroke in the bottom right) turns out to be not so simple to express algorithmically. When you try to make such rules precise, you quickly get lost in a morass of exceptions and caveats and special cases so that it seems hopeless.

Neural networks approach the problem in a different way. The idea is to take a large number of handwritten digits and then develop a system which can learn from those. 

By increasing the number of training examples, the network can learn more and more about handwriting, and so improve its accuracy. So while it has been shown just 100 training digits above, we could certainly build a better handwriting recognizer by using thousands or even millions or billions of training examples (**as we have seen above neural nets are not capable of extrapolating results, hence it won't recongnize a digit written in some strange way not included in the training sample !!!**).

Let's first try to implement a NN that is capable of recognizing handwritten digits.
In this example we will use tha sample provided with the $\tt{mnist}$ module.

Our program will be based on a slightly different kind of neural network than before, one type specifically designed for image/pattern recognition, the Convolutional Neural Network (CNN). We won't go into the details of its implementation since it is outside the scope of these lectures but it works essentially by applying on top of an image a series of filters (*convolutional layers*) that works as edge detectors. With them it classifies the images according to their relevant features.

Convolutional layers prove very effective, and stacking them allows to learn low-level features (e.g. lines) and high-order (more abstract) features, like shapes or specific objects.

<img src="edges.jpg">

Another important difference with respect to the previous examples is that in this case we are going to solve a classification problem (contrary to before when we were trying to regress a sample). Indeed our NN output won't be a single number but rather a list containing the probabilties that an image belonged to each class.

In [61]:
import numpy as np, mnist
from finnn import FinNN

# the actual images
train_images = mnist.train_images() 
# the target (it is a 0, 1, 2...)
train_labels = mnist.train_labels() 

# no test_size option means not split the sample
# in training and testing sets
# (MNIST has already dont it for us)
trainer = FinNN("CNN2D")
trainer.setData(train_images, train_labels)

Next we define the CNN architecture.

In [62]:
# define our convolutional NN
# we decide to apply 8 filters to the images 
# each with 3x3 pixels size
# the input images have 28x28 pixels size instead
trainer.addConv2DLayer(filters=8, filter_size=3, 
                       input_shape=(28, 28, 1))
trainer.addMaxPooling2D(2)
trainer.addFlatten()
trainer.addCNNOutputLayer(outputs=10)
        
trainer.compileModel(loss='categorical_crossentropy', 
                     opt='adam')

trainer.fit(epochs=5, verbose=1)
trainer.saveModel('digit_training')

8999
Epoch 1/5
59999/59999 [==============================] - 14s 235us/step - loss: 1.7046
Epoch 2/5
59999/59999 [==============================] - 14s 225us/step - loss: 0.3980
Epoch 3/5
59999/59999 [==============================] - 13s 217us/step - loss: 0.2738
Epoch 4/5
59999/59999 [==============================] - 14s 232us/step - loss: 0.2209
Epoch 5/5
59999/59999 [==============================] - 14s 229us/step - loss: 0.2026


In [63]:
trainer.loadModel('digit_training')

# testing with mnist test sample
test_images = mnist.test_images()
test_labels = mnist.test_labels()

trainer.setTestData(test_images, test_labels)
predictions = trainer.predict(trainer.x_test[:10])

print ("Tesing on MNIST digits...")
print("Predicted: ", np.argmax(predictions, axis=1)) 
print("Truth:", test_labels[:10])

# this line returns the highest probability of the vector
print("highest prob.:", ["{:.6f}".format(p[np.argmax(p)]) for p in predictions])

Tesing on MNIST digits...
Predicted:  [7 2 1 0 4 1 4 9 6 9]
Truth: [7 2 1 0 4 1 4 9 5 9]
highest prob.: ['1.000000', '0.999968', '0.999979', '1.000000', '1.000000', '0.999129', '0.999999', '0.994196', '0.997940', '1.000000']


Since the last but one digit has lower probability let's check the returned list to see which other number have non-zero probability.

In [64]:
print("9th digit:", ["dig {}: {:.6f}".format(i, p) for i, p in enumerate(predictions[8])])

9th digit: ['dig 0: 0.000000', 'dig 1: 0.000000', 'dig 2: 0.000000', 'dig 3: 0.000000', 'dig 4: 0.000000', 'dig 5: 0.002060', 'dig 6: 0.997940', 'dig 7: 0.000000', 'dig 8: 0.000000', 'dig 9: 0.000000']


So the second ranked digit is a 6 (which can be confused with a five if the lower loop is almost closed). 

To see how well our NN behaves with different kind of digits we will try to check how it works with my calligraphy (as homework try to repeat the exercise using your own).

Before passing the image to the NN it has to be resized and this is done with an ad-hoc function ($\tt{transform\_image}$) which is in the [$\tt{digit\_converter.py}$](https://drive.google.com/file/d/1FMYvOJDDOdIv7kDb2VIGhAkNNmReiOb_/view?usp=sharing) module.

In [65]:
from digit_converter import transform_image

filenames = ['four.png', 'five.png']

for f in filenames:
    test_images = np.array(transform_image(f))
    test_images = np.expand_dims(test_images, axis=3)

    predict = trainer.predict(test_images)
    print ("\n")
    print ("Tesing on custom digits...")
    print ("Predicted: ", np.argmax(predict, axis=1))
    print("%:", ["{:.3f}".format(p[np.argmax(p)]) for p in predict])
    print(["{:.2f}".format(p) for p in predict[0]])



Tesing on custom digits...
Predicted:  [4]
%: ['1.000']
['0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00', '0.00']


Tesing on custom digits...
Predicted:  [5]
%: ['1.000']
['0.00', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0.00', '0.00']


Those the images I have checked:

<img src="four.png" width=80>
<img src="five.png" width=80>

## Model Calibration cont.

When the parameter(s) of our model we need to calibrate can be expressed as a function of three variables the CNN can be used. Let's see how. 

Consider again the Black and Scholes formula for the call options. Assume you need to calibrate the rate $r$ and the volatity $\sigma$ at the same time. A convolutional neural network can be trained using special images which represents $\mathrm{ttm}, K$ and $P_\textrm{call}$.

A black-white image indeed can be interpreted as a map where each pixel is a pair ($\mathrm{ttm}, K)$ and the pixel color, an integer between 0 (black) and 255 (white), represents $P_\textrm{call}$. As in the previous examples the neural network was classifing the pictures into digits, now it will assign them to classes identified by $r, \sigma$ pairs.

The creation of the training sample is a little more complicated now. For convenience we will use also a new format to save data image, $\tt{numpy}$. This will be done through the corresponding module simply using the functions $\tt{save}$ and $\tt{load}$ to store and retrieve data.
The module $\tt{PIL}$ (pillow) is instead used to visualize the images.

First we make the targets.

In [27]:
import numpy as np
from finmarkets import call

labels = []
rates = np.arange(0.01, 0.11, 0.001)
vols = np.arange(0.1, 0.6, 0.005)
for i in range(len(vols)):
    for j in range(len(rates)):
        labels.append((vols[i], rates[j])) 
        
np.save("2d.np", labels)

Then we can create the images.

In [26]:
k = np.arange(0.8, 1.2, (1.2-0.8)/20)
ttm = np.arange(1, 5, 4/20)

# for each r, sigma pair
# generate a matrix of prices
maximum = 0
minimum = np.inf
prices = []
for v in vols:
    for r in rates:
        price =  np.zeros(shape=(20, 20))
        for ik, kv in enumerate(k):
            for it, t in enumerate(ttm):
                price[ik, it] = call(kv, r, v, t)
        prices.append(price)
        # max and min are saved to 
        # normalize our matrices
        new_max = np.max(price)
        new_min = np.min(price)
        if new_max > maximum:
            maximum = new_max
        if new_min < minimum:
            minimum = new_min
            
for ip, p in enumerate(prices):
    prices[ip] = np.interp(p, (minimum, maximum), (0, 1))

np.save("2d", prices)

Below an example of the 20x20 images that have been created.

<img src="2d_training_images.png" width=200>

Then the training is similar to what has been done for the handwritten digits.

In [71]:
import numpy as np
from finnn import FinNN

labels = np.load("2d_labels.npy")
images = np.load("2d.npy")

trainer = FinNN("CNN2D")
trainer.setData(images, labels, test_size=0.2)

trainer.addConv2DLayer(filters=8, filter_size=10, 
                       input_shape=(20, 20, 1), activation='relu')
trainer.addFlatten()
trainer.addHiddenLayer(neurons=10, activation='relu')
trainer.addOutputLayer(outputs=2, activation='relu')
trainer.compileModel(loss='mse', opt='adam')

trainer.fit(epochs=500, verbose=1)
trainer.saveModel("2d")

trainer.evaluate()

1200
Epoch 1/500
8000/8000 [==============================] - 1s 177us/step - loss: 0.0030
Epoch 2/500
8000/8000 [==============================] - 1s 164us/step - loss: 0.0011
Epoch 3/500
8000/8000 [==============================] - 1s 166us/step - loss: 4.8047e-04
Epoch 4/500
8000/8000 [==============================] - 1s 176us/step - loss: 2.3205e-04
Epoch 5/500
8000/8000 [==============================] - 1s 169us/step - loss: 1.6714e-04
Epoch 6/500
8000/8000 [==============================] - 1s 167us/step - loss: 9.7261e-05
Epoch 7/500
8000/8000 [==============================] - 1s 164us/step - loss: 1.2044e-04
Epoch 8/500
8000/8000 [==============================] - 1s 164us/step - loss: 1.1971e-04
Epoch 9/500
8000/8000 [==============================] - 1s 166us/step - loss: 8.1182e-05
Epoch 10/500
8000/8000 [==============================] - 1s 167us/step - loss: 8.3238e-05
Epoch 11/500
8000/8000 [==============================] - 1s 168us/step - loss: 5.8177e-05
Epoch 12/50

8000/8000 [==============================] - 1s 151us/step - loss: 3.0193e-05
Epoch 92/500
8000/8000 [==============================] - 1s 146us/step - loss: 5.7348e-06
Epoch 93/500
8000/8000 [==============================] - 1s 139us/step - loss: 7.0989e-06
Epoch 94/500
8000/8000 [==============================] - 1s 143us/step - loss: 1.1601e-05
Epoch 95/500
8000/8000 [==============================] - 1s 139us/step - loss: 6.2173e-06
Epoch 96/500
8000/8000 [==============================] - 1s 140us/step - loss: 1.0320e-05
Epoch 97/500
8000/8000 [==============================] - 1s 143us/step - loss: 7.3142e-06
Epoch 98/500
8000/8000 [==============================] - 1s 141us/step - loss: 1.0639e-05
Epoch 99/500
8000/8000 [==============================] - 1s 141us/step - loss: 9.6890e-06
Epoch 100/500
8000/8000 [==============================] - 1s 140us/step - loss: 2.4043e-06
Epoch 101/500
8000/8000 [==============================] - 1s 143us/step - loss: 1.5536e-05
Epoch 102/

8000/8000 [==============================] - 1s 142us/step - loss: 5.6513e-06
Epoch 181/500
8000/8000 [==============================] - 1s 143us/step - loss: 2.5952e-05
Epoch 182/500
8000/8000 [==============================] - 1s 143us/step - loss: 1.8422e-06
Epoch 183/500
8000/8000 [==============================] - 1s 141us/step - loss: 1.8579e-06
Epoch 184/500
8000/8000 [==============================] - 1s 142us/step - loss: 7.5490e-06
Epoch 185/500
8000/8000 [==============================] - 1s 141us/step - loss: 6.1259e-06
Epoch 186/500
8000/8000 [==============================] - 1s 139us/step - loss: 5.4931e-05
Epoch 187/500
8000/8000 [==============================] - 1s 137us/step - loss: 4.2762e-06
Epoch 188/500
8000/8000 [==============================] - 1s 137us/step - loss: 2.2109e-06
Epoch 189/500
8000/8000 [==============================] - 1s 143us/step - loss: 9.3376e-06
Epoch 190/500
8000/8000 [==============================] - 1s 142us/step - loss: 1.6687e-06
Ep

8000/8000 [==============================] - 1s 154us/step - loss: 5.1353e-06
Epoch 270/500
8000/8000 [==============================] - 1s 172us/step - loss: 9.8053e-06
Epoch 271/500
8000/8000 [==============================] - 1s 149us/step - loss: 3.3140e-06
Epoch 272/500
8000/8000 [==============================] - 1s 150us/step - loss: 2.7765e-06
Epoch 273/500
8000/8000 [==============================] - 1s 166us/step - loss: 5.7508e-06
Epoch 274/500
8000/8000 [==============================] - 2s 208us/step - loss: 2.7988e-06
Epoch 275/500
8000/8000 [==============================] - 2s 194us/step - loss: 4.1211e-06
Epoch 276/500
8000/8000 [==============================] - 1s 146us/step - loss: 4.9778e-06
Epoch 277/500
8000/8000 [==============================] - 1s 148us/step - loss: 8.7270e-06
Epoch 278/500
8000/8000 [==============================] - 1s 162us/step - loss: 8.8475e-06
Epoch 279/500
8000/8000 [==============================] - 2s 195us/step - loss: 4.1896e-06
Ep

8000/8000 [==============================] - 1s 151us/step - loss: 5.8279e-06
Epoch 359/500
8000/8000 [==============================] - 1s 144us/step - loss: 2.8505e-06
Epoch 360/500
8000/8000 [==============================] - 1s 146us/step - loss: 2.5505e-06
Epoch 361/500
8000/8000 [==============================] - 1s 150us/step - loss: 6.4110e-06
Epoch 362/500
8000/8000 [==============================] - 1s 143us/step - loss: 5.5498e-06
Epoch 363/500
8000/8000 [==============================] - 1s 144us/step - loss: 1.8211e-06
Epoch 364/500
8000/8000 [==============================] - 1s 150us/step - loss: 6.8583e-06
Epoch 365/500
8000/8000 [==============================] - 1s 142us/step - loss: 5.5341e-06
Epoch 366/500
8000/8000 [==============================] - 1s 144us/step - loss: 1.3020e-06
Epoch 367/500
8000/8000 [==============================] - 1s 145us/step - loss: 3.1088e-06
Epoch 368/500
8000/8000 [==============================] - 1s 143us/step - loss: 4.9312e-06
Ep

8000/8000 [==============================] - 1s 155us/step - loss: 3.6593e-06
Epoch 448/500
8000/8000 [==============================] - 1s 149us/step - loss: 5.8372e-06
Epoch 449/500
8000/8000 [==============================] - 1s 146us/step - loss: 2.7543e-06
Epoch 450/500
8000/8000 [==============================] - 1s 151us/step - loss: 3.8940e-06
Epoch 451/500
8000/8000 [==============================] - 1s 151us/step - loss: 3.7809e-06
Epoch 452/500
8000/8000 [==============================] - 1s 154us/step - loss: 4.2284e-06
Epoch 453/500
8000/8000 [==============================] - 1s 148us/step - loss: 2.4083e-06
Epoch 454/500
8000/8000 [==============================] - 1s 142us/step - loss: 3.1988e-06
Epoch 455/500
8000/8000 [==============================] - 1s 152us/step - loss: 6.2170e-06
Epoch 456/500
8000/8000 [==============================] - 1s 153us/step - loss: 2.6136e-06
Epoch 457/500
8000/8000 [==============================] - 1s 150us/step - loss: 1.5417e-06
Ep

At this point you should present to the trained CNN the prices of call referring to the same underlying in the pictorial form shown before and the in response it will give you the risk-free rate and the underlying volatility.

In [72]:
for i in range(5):
    print (trainer.predict(trainer.x_test[i:i+1]))

[[0.2726415  0.01990955]]
[[0.5474939  0.08093072]]
[[0.30228412 0.10630756]]
[[0.44526297 0.08495573]]
[[0.15819341 0.03270075]]


### Technical Analysis

In finance, *technical analysis* is a security analysis discipline for forecasting the direction of prices through the study of past market data, primarily price and volume.
Essentially the analyst looks for particular patterns in the price time series that are *known* to develop in predictable ways to take profit of it.

<img src="H_and_s_top_new.jpg" width=400>
<img src="Triangle-ascending.jpg" width=400>

As you may imagine we will try to develop a CNN (like in the handwriting case) capable of classifying features in time series to be used in a technical analysis (this is much faster than having somebody looking at thousands of time series by eye...).

I have generated myself the training set simulating 21600 time series (1/3 with head and shoulder patter, 1/3 with triangle pattern and 1/3 with no pattern). *To make the training easier the features have been exagerated.*

<img src="image_1.png" width=300>
<img src="image_2.png" width=300>
<img src="image_0.png" width=300>

In [73]:
import numpy as np
from finnn import FinNN

train_labels = np.load("training_techana_labels.npy")
train_images = np.load("training_techana_images.npy")

trainer = FinNN("CNN1D")
trainer.setData(train_images, train_labels)

# define the CNN 
trainer.addConv1DInputLayer(filters=80, filter_size=20, 
                            input_size=(101, 1))
trainer.addConv1DLayer(filters=80, filter_size=15)
trainer.addMaxPooling1D(3)
trainer.addConv1DLayer(filters=100, filter_size=10)
trainer.addConv1DLayer(filters=100, filter_size=5)
trainer.addGlobalAveragePooling1D()
trainer.addDropout(0.5)
trainer.addCNNOutputLayer(outputs=3)

trainer.compileModel(loss='categorical_crossentropy', 
                     opt='adam')

# make the training
trainer.fit(epochs=80)

trainer.saveModel('techana')

3239
Epoch 1/80
21599/21599 [==============================] - 26s 1ms/step - loss: 0.6753
Epoch 2/80
21599/21599 [==============================] - 26s 1ms/step - loss: 0.5232
Epoch 3/80
21599/21599 [==============================] - 26s 1ms/step - loss: 0.4710
Epoch 4/80
21599/21599 [==============================] - 26s 1ms/step - loss: 0.4118
Epoch 5/80
21599/21599 [==============================] - 25s 1ms/step - loss: 0.3852
Epoch 6/80
21599/21599 [==============================] - 26s 1ms/step - loss: 0.3607
Epoch 7/80
21599/21599 [==============================] - 25s 1ms/step - loss: 0.3173
Epoch 8/80
21599/21599 [==============================] - 26s 1ms/step - loss: 0.2778
Epoch 9/80
21599/21599 [==============================] - 26s 1ms/step - loss: 0.2513
Epoch 10/80
21599/21599 [==============================] - 26s 1ms/step - loss: 0.2336
Epoch 11/80
21599/21599 [==============================] - 27s 1ms/step - loss: 0.2148
Epoch 12/80
21599/21599 [======================

To test the perfomance I wanted to simulate a real case scenario where the time series are analyzed in real-time in order to predict as soon as possible a particular pattern and take advantage of the prediction.

To do so I have created a longer time series (i.e. more time points) and passed as input to the CNN sliding time windows to simulate the evolution of the time series. So if for example the time series is made of 100 points, I presented to the network first the points between $[0, 80]$, then $[1, 81]$, $[2, 82]$ and so on simulating new real time data incoming. 
The goal was to check when the neural network was capable of predicting the incoming pattern.

<img src="closing_price.gif">

In [74]:
test_images = np.load("testing_techana_frames.npy")
trainer.loadModel("techana")

predictions = trainer.predict(test_images)
for i in range(len(predictions)):
    print (np.argmax(predictions[i]), ["{:.3f}".format(p) for p in predictions[i]])

0 ['0.898', '0.000', '0.102']
0 ['0.833', '0.000', '0.167']
0 ['1.000', '0.000', '0.000']
0 ['0.999', '0.000', '0.000']
0 ['0.952', '0.048', '0.000']
1 ['0.270', '0.730', '0.000']
1 ['0.000', '1.000', '0.000']
1 ['0.000', '1.000', '0.000']
1 ['0.000', '1.000', '0.000']
1 ['0.000', '1.000', '0.000']


So at the 6th sample the CNN start recognizing the *head and shoulder* pattern in the price evolution.